In [1]:
import shadie

## Make Custom Mutations
First, we create two mutation types with differential expression - one mutation type will only affect the haploid life stage and the other will only affect the diploid life stage. 

In [36]:
m3 = shadie.mtype(0.5, 'f', 0.5, diffexpr="haploid") #introduced beneficial mutation in haploids
m4 = shadie.mtype(0.5, 'f', 0.1, diffexpr="diploid") #introduced beneficial mutation in diploids
NEUT = shadie.mtype(0.5, "f", 0.0,) #neutral mutations in the rest of the chromosome

## Make Custom Genomic Elements
Here, we create two small genomic elements. One will only be expressed duriing the haploid life stage and one during the diploid life stage - this functionality is a result of the mutation types used when defining the genomic element. 

In [37]:
e1 = shadie.etype([NEUT, m3, m4], [1,0,0]) #inherits m1 __; only expressed in haploids

## Build the Chromosome
Finally, we use our genomic elements to define genes on our chromosome. Each is 1000 base pairs long. 

In [5]:
chrom = shadie.chromosome.explicit(data = {(0, 5000000): shadie.EMPTY,
                                           (5000001, 5001000):shadie.NONCDS,
                                           (5001001, 10000000): shadie.EMPTY,
                                           (10000001, 10001000):shadie.NONCDS,
                                           (10001001, 15000000): shadie.EMPTY},
                                   use_synonymous_sites_in_coding=False,
                                   )

In [38]:
chrom = shadie.chromosome.explicit(data = {(0, 1000000): e1},
                                   use_synonymous_sites_in_coding=False,
                                   )

In [39]:
#when we visualiize our chromosome we can see two tiny genes at 5e6 and 10e6
chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t333463de2a754962909fbd2284c4d9b1"> name: None
interval: (0, 1000000)
ElementType: g3
coding: True 0 200000 400000 600000 800000 1000000

## Create the Simulation 
This simulation will run for X generations, then add a single beneficial mutation (selection coefficient = 0.5) to the gene expressed in the gametophyte and a single beneficial mutation (selection coefficient = 0.5; dominance coefficient = 0.5) to the gene expressed in the sporophyte. 

This simulation was initially included in Sorojsrisom et al. (2022), alongside a more complex simulation investigating pattens of diversity across the genome resulting from targeted differential selection on each life stage (see this other jupyter notebook). For simplicity in the manuscript, we use the same chromosome structure for both simulations, but single-site simulations can be implemented on a much simpler chromosome (see this other tutorial). 

In [45]:
# These are the SLiM scripts that will add a mutation at a given time. 

add_mut_gam = """
1000 late() {
    num_genomes = length(p0.genomes);
	list_genomes = 1:num_genomes ;
	even_genomes= (p0.genomes)[list_genomes % 2 == 1];
	target = sample(even_genomes, 8);
	target.addNewDrawnMutation(m3, 500000);
}
"""

add_mut_spo = """
1001 late() {
    target = sample(p1.genomes, 10);
    target.addNewDrawnMutation(m4, 500000);
}
"""

In [32]:
with shadie.Model() as bryomono_gam:
    # init the model
    bryomono_model.initialize(chromosome=chrom, 
                     sim_time = 3000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_out = "singlemutgam-bryo-mono.trees")
    
    # add custom script which adds a single beneficial mutation 
    # to each gene at t=6000
    bryomono_model.custom(
        scripts=add_mut_gam,
        comment = "add beneficial mutation to gametophytes at t=1000")
        
    # set model parameters 
    bryomono_model.reproduction.bryophyte_monoicous(
        spo_pop_size=500, 
        gam_pop_size=500,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,)
    
#Uncomment line below to view the script
print(bryomono_model.script)


initialize() {

  // model type
  initializeSLiMModelType("nonWF");

  // config
  initializeRecombinationRate(1e-07, 1000000);
  initializeMutationRate(1e-06);
  initializeTreeSeq(simplificationInterval=200);

  // MutationType init
  initializeMutationType('m3', 0.5, 'f', 0.1);
    m3.convertToSubstitution = T;
  initializeMutationType('m5', 0.5, 'f', 0.0);
    m5.convertToSubstitution = T;
  c(m5, m3).haploidDominanceCoeff = 1.0;

  // ElementType init
  initializeGenomicElementType('g1', c(m5,m3), c(1,0));

  // Chromosome (GenomicElement init)
  initializeGenomicElement(g1, 0, 1000000);

  // constants (Population sizes and others)
  defineConstant('SPO_POP_SIZE', 500);
  defineConstant('GAM_POP_SIZE', 500);
  defineConstant('SPO_MUTATION_RATE', 5e-07);
  defineConstant('GAM_MUTATION_RATE', 5e-07);
  defineConstant('GAM_CLONE_RATE', 0.2);
  defineConstant('GAM_CLONES_PER', 1);
  defineConstant('GAM_SELF_RATE_PER_EGG', 0.0);
  defineConstant('GAM_SELF_RATE', 0.2);
  defineConstant

In [46]:
with shadie.Model() as bryomono_spo:
    # init the model
    bryomono_spo.initialize(chromosome=chrom, 
                     sim_time = 3000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_out = "singlemutspo-bryo-mono.trees")
    
    # add custom script which adds a single beneficial mutation 
    # to each gene at t=6000
    bryomono_spo.custom(
        scripts=add_mut_spo,
        comment = "add beneficial mutation to gametophytes at t=1000")
        
    # set model parameters 
    bryomono_spo.reproduction.bryophyte_monoicous(
        spo_pop_size=500, 
        gam_pop_size=500,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,)
    
#Uncomment line below to view the script
#print(bryomono_spo.script)

In [47]:
bryomono_spo.run()

In [ ]:
## Using the `PostSim` Module

In [48]:
import pyslim

In [49]:
bryomono_spo = "singlemutspo-bryo-mono.trees"

In [51]:
post1 = shadie.postsim.OneSim(
    bryomono_spo,
    chromosome=chrom, 
    mut=5e-7,
    recomb=1e-7, 
    recapitate=False, 
    ancestral_Ne=3000, 
    add_neutral_mutations=False,
)

post1.tree_sequence
#post.draw_tree_sequence(max_trees=10);

/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +


In [53]:
post1.draw_tree_sequence(max_trees=10);

⚠️ toytree: tree has multiple (2) roots.
⚠️ toytree: tree has multiple (2) roots.
⚠️ toytree: tree has multiple (2) roots.
⚠️ toytree: tree has multiple (2) roots.


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1500.0px" height="425.0px" viewBox="0 0 1500.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9cdbf7d1a7ad4aeb8f2ce193fb74b424"> name: None
interval: (0, 1000000)
ElementType: g3
coding: True 0 200000 400000 600000 800000 1000000 idx: 0
interval: (0 - 10768)
tmrca: 3000.0 idx: 1
interval: (10768 - 13690)
tmrca: 3000.0 <polygon points="289.5580131590669,108.76760767491737 337.5,143.83803837458703 337.5,319.19019187293486 474.99999999999994,319.19019187293486 474.99999999999994,143.83803837458703 436.23261259656135,108.76760767491737 436.23261259656135,108.76760767491737 474.99999999999994,108.76760767491737 474.99999999999994,108.76760767491737 337.5,108.76760767491737 337.5,108.76760767491737 289.5580131590669,108.76760767491737" style="fill:rgb(55.3%,62.7%,79.6%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 2
interval: (13690 - 22072)
tmrca: 3000.0 <polygon points="436.23261259656135,108.76760767491737 474.99999999999994,143.83803837458703 474.99999999999994,319.19019187293486 612.5000000000001,319.19019187293486 612.5000000000001,143.83803837458703 443.45960013744474,108.76760767491737 443.45960013744474,108.76760767491737 612.5000000000001,108.76760767491737 612.5000000000001,108.76760767491737 474.99999999999994,108.76760767491737 474.99999999999994,108.76760767491737 436.23261259656135,108.76760767491737" style="fill:rgb(90.6%,54.1%,76.5%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 3
interval: (22072 - 22485)
tmrca: 3000.0 <polygon points="443.45960013744474,108.76760767491737 612.5000000000001,143.83803837458703 612.5000000000001,319.19019187293486 750.0,319.19019187293486 750.0,143.83803837458703 694.8292757422656,108.76760767491737 694.8292757422656,108.76760767491737 750.0,108.76760767491737 750.0,108.76760767491737 612.5000000000001,108.76760767491737 612.5000000000001,108.76760767491737 443.45960013744474,108.76760767491737" style="fill:rgb(65.1%,84.7%,32.9%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 4
interval: (22485 - 36850)
tmrca: 3000.0 <polygon points="694.8292757422656,108.76760767491737 750.0,143.83803837458703 750.0,319.19019187293486 887.4999999999999,319.19019187293486 887.4999999999999,143.83803837458703 716.667727197526,108.76760767491737 716.667727197526,108.76760767491737 887.4999999999999,108.76760767491737 887.4999999999999,108.76760767491737 750.0,108.76760767491737 750.0,108.76760767491737 694.8292757422656,108.76760767491737" style="fill:rgb(100%,85.1%,18.4%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 5
interval: (36850 - 38098)
tmrca: 3000.0 <polygon points="716.667727197526,108.76760767491737 887.4999999999999,143.83803837458703 887.4999999999999,319.19019187293486 1025.0,319.19019187293486 1025.0,143.83803837458703 863.7622968553139,108.76760767491737 863.7622968553139,108.76760767491737 1025.0,108.76760767491737 1025.0,108.76760767491737 887.4999999999999,108.76760767491737 887.4999999999999,108.76760767491737 716.667727197526,108.76760767491737" style="fill:rgb(89.8%,76.9%,58%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 6
interval: (38098 - 46504)
tmrca: 3000.0 idx: 7
interval: (46504 - 47733)
tmrca: 3000.0 idx: 8
interval: (47733 - 71406)
tmrca: 3000.0 idx: 9
interval: (71406 - 78577)
tmrca: 3000.0 p0-5 p0-1 p0-2 p0-9 p0-4 p0-7 p0-3 p0-6 p0-8 p0-0 id: 0
pos: 0
time: 1785
mtype: 6 id: 1
pos: 1
time: 1463
mtype: 6 id: 2
pos: 2
time: 765
mtype: 6 id: 3
pos: 3
time: 137
mtype: 6 id: 4
po

In [9]:
# These are the SLiM scripts that will add a mutation at a given time. 

add_mut_gam = """
6000 late() {
    num_genomes = length(p0.genomes);
	list_genomes = 1:num_genomes ;
	even_genomes= (p0.genomes)[list_genomes % 2 == 1];
	target = sample(even_genomes, 8);
	target.addNewDrawnMutation(m4, 5000500);
}
"""

add_mut_spo = """
6000 late() {
    target = sample(p1.genomes, 1);
    target.addNewDrawnMutation(m4, 10000500);
}
"""

In [12]:
with shadie.Model() as bryomono_model:
    # init the model
    bryomono_model.initialize(chromosome=chrom, 
                     sim_time = 10000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_out = "singlemutgam-bryo-mono.trees")
    
    # add custom script which adds a single beneficial mutation 
    # to each gene at t=6000
    bryomono_model.custom(
        scripts=add_mut_gam,
        comment = "add beneficial mutation to gametophytes at t=6000")
        
    # set model parameters 
    bryomono_model.reproduction.bryophyte_monoicous(
        spo_pop_size=3000, 
        gam_pop_size=3000,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,)
    
#Uncomment line below to view the script
print(bryomono_model.script)


initialize() {

  // model type
  initializeSLiMModelType("nonWF");

  // config
  initializeRecombinationRate(1e-07, 15000000);
  initializeMutationRate(1e-06);
  initializeTreeSeq(simplificationInterval=200);

  // MutationType init
  initializeMutationType('m1', 0.5, 'f', 0.0);
    m1.convertToSubstitution = T;
  initializeMutationType('m4', 0.5, 'f', 0.1);
    m4.convertToSubstitution = T;
  initializeMutationType('m5', 0.5, 'f', 0.1);
    m5.convertToSubstitution = T;
  c(m5, m4, m1).haploidDominanceCoeff = 1.0;

  // ElementType init
  initializeGenomicElementType('g3', m1, 1);
  initializeGenomicElementType('g5', m4, 1);
  initializeGenomicElementType('g6', m5, 1);

  // Chromosome (GenomicElement init)
  initializeGenomicElement(g3, 0, 5000000);
  initializeGenomicElement(g5, 5000001, 5001000);
  initializeGenomicElement(g3, 5001001, 10000000);
  initializeGenomicElement(g6, 10000001, 10001000);
  initializeGenomicElement(g3, 10001001, 15000000);

  // constants (Population si